## Setup
### Import

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import os
import sys
from datetime import datetime, timedelta

# Set up matplotlib for nicer plots
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# Add parent directory to path for imports
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
# Import data utilities
from utils.data_utils import fetch_data, preprocess_data, resample_data, get_risk_free_rate

# Import indicator utilities
from utils.indicator_utils import calculate_kdj, calculate_sma, calculate_macd, calculate_rsi

# Import performance utilities
from utils.performance_utils import (
    calculate_returns, plot_cumulative_returns, 
    plot_rolling_metrics, plot_drawdowns, generate_performance_report
)

# Import strategy
from strategies.kdj_strategy import KDJStrategy

# Import backtesting engine
from backtest.engine import BacktestEngine

# Import optimization module
from optimize.grid_search import GridSearch

### Configure Parameters

In [6]:
# Configuration
ticker = ''           # Ticker symbol
benchmark = 'SPY'         # Benchmark ticker
period = '4y'            # Data period (e.g., '1y', '5y', '10y')
initial_capital = 100000  # Initial capital for backtesting
#start_date = datetime.now()         # Start date for backtesting
#end_date = datetime.now()           # End date for backtesting

# Strategy parameters
strategy_params = {
    'k_period': 9,           
    'j_buy_threshold': -5,   
    'j_sell_threshold': 100,  
    'daily_enabled': True,   
    'weekly_enabled': True,  
    'monthly_enabled': False,
    'position_size': 0.7     
}

### Download and Preprocess Data

In [ ]:
# Fetch data

print(f"Fetching data for {ticker} with {benchmark} as benchmark...")
data = fetch_data(ticker, period=period, benchmark=benchmark)


In [ ]:
print(data.head())

In [ ]:


# Preprocess data
data = preprocess_data(data, ticker)

# Display first few rows
print(data.head())

## Strategy Implementation

In [ ]:
# Create strategy instance
strategy = KDJStrategy(ticker, strategy_params)

# Prepare data for the strategy
prepared_data = strategy.prepare_data(data)

# Generate trading signals
signals_data = strategy.generate_signals(prepared_data)

# Display generated signals
signals_data[[f'{ticker}_close', f'{ticker}_kdj_j_weighted', f'{ticker}_signal', f'{ticker}_position_size']].tail(10)

In [ ]:
# Define function to plot KDJ indicator with signals
def plot_kdj_signals(df, ticker, start_date=None, end_date=None, figsize=(12, 8), strategy_params=strategy_params):
    if start_date:
        df = df.loc[start_date:]
    if end_date:
        df = df.loc[:end_date]
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, sharex=True)

    # Plot price with buy/sell signals
    ax1.plot(df.index, df[f'{ticker}_close'], label='Close Price')
    buy_signals = df[df[f'{ticker}_signal'] == 1]
    sell_signals = df[df[f'{ticker}_signal'] == -1]

    ax1.scatter(buy_signals.index, buy_signals[f'{ticker}_close'], color='green', marker='^', s=100, label='Buy Signal')
    ax1.scatter(sell_signals.index, sell_signals[f'{ticker}_close'], color='red', marker='v', s=100, label='Sell Signal')

    ax1.set_title(f'{ticker} Price and Signals')
    ax1.set_ylabel('Price')
    ax1.legend()
    ax1.grid(True)

    # Plot KDJ J line with thresholds
    ax2.plot(df.index, df[f'{ticker}_kdj_j_weighted'], label='J weighted', color='purple')
    ax2.axhline(y=strategy_params['j_buy_threshold'], color='green', linestyle='--', label='Buy Threshold')
    ax2.axhline(y=strategy_params['j_sell_threshold'], color='red', linestyle='--', label='Sell Threshold')

    ax2.set_ylabel('KDJ J Value')
    ax2.set_xlabel('Date')
    ax2.legend()
    ax2.grid(True)

    fig.tight_layout()
    return fig

# Plot last 180 days
end_date = signals_data.index[-1]
start_date = signals_data.index[0]
plot_kdj_signals(signals_data, ticker, start_date, end_date)

## Backtesting

In [ ]:
# Get start and end dates from our data
start_date = data.index[0]
end_date = data.index[-1]

# Fetch daily risk-free rates for the entire period
print(f"Fetching daily risk-free rates...")
risk_free_rates = get_risk_free_rate(start_date=start_date, end_date=end_date)
print(risk_free_rates.head())

In [ ]:
# Create backtesting engine with daily risk-free rates
engine = BacktestEngine(initial_capital=initial_capital, risk_free_rate=risk_free_rates)

# Run backtest
print(f"Running backtest for {ticker} with strategy {strategy.name}...")
backtest_results = engine.run(data, ticker, strategy)

# Display backtest results
print(backtest_results[[f'{ticker}_position', f'{ticker}_cash', f'{ticker}_equity', f'{ticker}_strategy_return']].tail())

In [53]:
portfolio_trade = engine.get_trades(backtest_results, ticker)

In [ ]:
print(portfolio_trade.columns)

In [ ]:
print(portfolio_trade.iloc[:,-12:])

In [ ]:
print(backtest_results.columns)

## Performance Analysis

In [ ]:
# Calculate performance metrics using daily risk-free rates
metrics = calculate_returns(backtest_results, ticker, benchmark, risk_free_rates)

# Generate performance report
generate_performance_report(metrics)

In [ ]:
engine.plot_equity_curve(backtest_results, ticker, benchmark)
plot_rolling_metrics(backtest_results, ticker)
engine.plot_drawdowns(backtest_results, ticker)

## Optimization

In [ ]:
# Define parameter grid
param_grid = {
    'k_period': [9],
    'j_buy_threshold': [-5, 0,5],
    'j_sell_threshold': [  100, 105, 110],
    'daily_enabled': [True],
    'weekly_enabled': [True],
    'position_size': [ 0.5,  0.7, 1.0],
    'weekly_weight': [ 0.5, 0.75, 1.0],
}

# Create and run grid search
grid_search = GridSearch(
    data=data,
    ticker=ticker,
    strategy_class=KDJStrategy,
    param_grid=param_grid,
    metric='sharpe_ratio',
    maximize=True,
    n_jobs=1  
)
grid_search.fit()

## Optimization Validation

In [ ]:
# Apply best parameters
best_strategy = KDJStrategy(ticker, grid_search.best_params)
print(f"Best parameters: {grid_search.best_params}")

# Run final backtest
best_backtest = engine.run(data, ticker, best_strategy)

# Evaluate performance
best_metrics = calculate_returns(best_backtest, ticker, benchmark)
generate_performance_report(best_metrics)

# Plot final equity curve
engine.plot_equity_curve(best_backtest, ticker, benchmark)

In [ ]:
best_signals_data = best_strategy.generate_signals(best_strategy.prepare_data(data))
plot_kdj_signals(best_signals_data, ticker, start_date, end_date, strategy_params=best_strategy.params)

In [ ]:
engine.plot_equity_curve(best_backtest, ticker, benchmark)
plot_rolling_metrics(best_backtest, ticker)
engine.plot_drawdowns(best_backtest, ticker)

$$
dG(t,F)
= \underbrace{\frac{\partial G}{\partial t}\,dt}{\text{time part}}
\;+\;\underbrace{\frac{\partial G}{\partial F}\,dF}{\text{linear part in } dF}
\;+\;\underbrace{\tfrac12 \,\frac{\partial^2 G}{\partial F^2}\,(dF)^2}_{\text{second-order term}}
\;+\;\dots
$$